### Some sites the last year's records are not full. 
### Here scan the file and fill the missing rows

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import datetime

In [2]:
src_dir = './output/'
qcv_dir = '/01_qc_visual/qcv_files/'

In [13]:
for sub_dir_list in os.walk(src_dir):
    break

sub_dir_list[1]

['US-xWR',
 'US-Syv',
 'US-CC2',
 'US-MC1',
 'US-Kon',
 'US-Dk3',
 'US-Bo2',
 'US-xJE',
 'US-CRT',
 'US-TrS',
 'US-xSJ',
 'US-Wi1',
 'US-DFK',
 'US-SRG',
 'US-Wlr',
 'US-DPW',
 'US-PFf',
 'CA-TP3',
 'US-MOz',
 'US-UiA',
 'US-Wi8',
 'US-GLE',
 'US-PSH',
 'US-PFh',
 'US-MtB',
 'US-CF2',
 'US-xST',
 'US-MVF',
 'US-xSL',
 'US-Jo1',
 'US-Ho3',
 'US-Mi3',
 'US-MMS',
 'CA-TP2',
 'US-Lin',
 'US-Wi3',
 'US-xSR',
 'US-Ne1',
 'US-Dia',
 'US-PHM',
 'US-PFa',
 'US-Ha2',
 'US-xDL',
 'US-RGo',
 'US-Pnp',
 'US-JRn',
 'US-Ho2',
 'US-Bi1',
 'CA-MA1',
 'US-Fuf',
 'US-MVW',
 'US-KFS',
 'US-Oho',
 'US-Mpj',
 'US-SP2',
 'US-Blo',
 'CA-DB2',
 'US-AR1',
 'US-Akn',
 'US-xJR',
 'US-PFd',
 'US-NR1',
 'US-CS5',
 'US-Vcp',
 'US-Rms',
 'US-Rwf',
 'US-xSP',
 'CA-MA3',
 'US-xUK',
 'CA-ARB',
 'US-TrB',
 'US-xUN',
 'US-xRM',
 'US-PFc',
 'MX-Aog',
 'US-Mj1',
 'US-CF4',
 'US-xBL',
 'US-DS3',
 'US-Vcm',
 'US-NC4',
 'US-Vcs',
 'US-Ro5',
 'US-Wi9',
 'US-Wrc',
 'US-CS3',
 'US-RGA',
 'US-xWD',
 'US-Tw3',
 'US-xSC',
 'US-ASM',

In [12]:
full_qcv_dir

'./output/US-RGW/01_qc_visual/qcv_files/'

In [17]:
last_year_qcv_file

'./output/US-xDS/01_qc_visual/qcv_files/US-xDS_qcv_2021.csv'

In [14]:
#add_row_files = []
for sub_dir in sub_dir_list[1]:
    site_name = sub_dir
    
    full_qcv_dir = src_dir + site_name + qcv_dir
    qcv_file_list = glob.glob(full_qcv_dir + '*.csv')
    years = []
    for qcv_file in qcv_file_list:
        qcv_name = os.path.basename(qcv_file)
        year = qcv_name.split('.')[0][-4:]
        years.append(int(year))
        
    years.sort()
    last_year = years[-1]
    last_year_qcv_file = full_qcv_dir + site_name + '_qcv_' + str(last_year) + '.csv'
    data = pd.read_csv(last_year_qcv_file, dtype={'TIMESTAMP_START':str, 'TIMESTAMP_END':str},skiprows=9)
    
    _year = data.iloc[-1]['TIMESTAMP_END'][:4]
    _year = int(_year)
    if _year > last_year: # the rows are full
        continue
    
    print(last_year_qcv_file, 'has rows {}'.format(data.index[-1]))
    start_day = '{:4d}01010000'.format(last_year)
    end_day = '{:4d}01010000'.format(last_year + 1)
    T = pd.date_range(start_day, end_day,freq='30min')
    new_index = pd.RangeIndex(T.size - data.index[-1]-2)
    new_df = pd.DataFrame(np.nan, index= new_index, columns=data.columns)
    start_t = data['TIMESTAMP_START'].count()
    j = 0
    for i in range(start_t, T.size-1):
        start_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i].year,T[i].month,T[i].day,T[i].hour, T[i].minute)
        end_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i+1].year,T[i+1].month,T[i+1].day,T[i+1].hour, T[i+1].minute)
        new_df.loc[j,'TIMESTAMP_START'] = start_time
        new_df.loc[j,'TIMESTAMP_END'] = end_time
        j+=1
        
    full_data = pd.concat([data,new_df], axis=0,ignore_index=True)
    full_data = full_data.interpolate()
    
    # read first 9 lines from the qcv csv file
    file1 = open(last_year_qcv_file, 'r')
    count = 0
    pre_lines = []
    for line in file1:
        count += 1
        #print("Line{}: {}".format(count, line.strip()))
        pre_lines.append(line.strip())
        if count == 9:
            break

    # Closing files
    file1.close()
    
    # update rows
    full_data.to_csv(last_year_qcv_file, index=False)

    file = open(last_year_qcv_file,'r')
    text = file.read()
    file.close()

    with open(last_year_qcv_file,'w') as f:
        f.write('\n'.join(pre_lines))
        f.write('\n')
        f.write(text)
        
    add_row_files.append(last_year_qcv_file)

./output/US-Rls/01_qc_visual/qcv_files/US-Rls_qcv_2020.csv has rows 13151
./output/US-CS4/01_qc_visual/qcv_files/US-CS4_qcv_2021.csv has rows 5567
./output/US-UiB/01_qc_visual/qcv_files/US-UiB_qcv_2016.csv has rows 17566
./output/US-LS2/01_qc_visual/qcv_files/US-LS2_qcv_2007.csv has rows 17518
./output/US-Mi1/01_qc_visual/qcv_files/US-Mi1_qcv_2019.csv has rows 17381
./output/US-PFb/01_qc_visual/qcv_files/US-PFb_qcv_2019.csv has rows 14591
./output/US-ASH/01_qc_visual/qcv_files/US-ASH_qcv_2017.csv has rows 4319
./output/US-PFm/01_qc_visual/qcv_files/US-PFm_qcv_2019.csv has rows 14591
./output/US-MH1/01_qc_visual/qcv_files/US-MH1_qcv_2015.csv has rows 17518
./output/US-PFq/01_qc_visual/qcv_files/US-PFq_qcv_2019.csv has rows 14591
./output/CA-ER1/01_qc_visual/qcv_files/CA-ER1_qcv_2021.csv has rows 5758
./output/US-LL2/01_qc_visual/qcv_files/US-LL2_qcv_2017.csv has rows 17518
./output/US-PFi/01_qc_visual/qcv_files/US-PFi_qcv_2019.csv has rows 14591
./output/US-Srr/01_qc_visual/qcv_files/US

In [18]:
with open('add_rows_files.txt', 'w') as f:
    for line in add_row_files:
        f.write('{}\n'.format(line))
        
print('Done!')

Done!


In [16]:
add_row_files

['./output/US-MC1/01_qc_visual/qcv_files/US-MC1_qcv_2015.csv',
 './output/US-Kon/01_qc_visual/qcv_files/US-Kon_qcv_2019.csv',
 './output/US-TrS/01_qc_visual/qcv_files/US-TrS_qcv_2020.csv',
 './output/US-DFK/01_qc_visual/qcv_files/US-DFK_qcv_2021.csv',
 './output/US-Wlr/01_qc_visual/qcv_files/US-Wlr_qcv_2004.csv',
 './output/US-DPW/01_qc_visual/qcv_files/US-DPW_qcv_2017.csv',
 './output/US-PFf/01_qc_visual/qcv_files/US-PFf_qcv_2019.csv',
 './output/US-Wi8/01_qc_visual/qcv_files/US-Wi8_qcv_2002.csv',
 './output/US-GLE/01_qc_visual/qcv_files/US-GLE_qcv_2020.csv',
 './output/US-PSH/01_qc_visual/qcv_files/US-PSH_qcv_2017.csv',
 './output/US-PFh/01_qc_visual/qcv_files/US-PFh_qcv_2019.csv',
 './output/US-CF2/01_qc_visual/qcv_files/US-CF2_qcv_2021.csv',
 './output/US-MVF/01_qc_visual/qcv_files/US-MVF_qcv_2017.csv',
 './output/US-Mi3/01_qc_visual/qcv_files/US-Mi3_qcv_2019.csv',
 './output/US-JRn/01_qc_visual/qcv_files/US-JRn_qcv_2019.csv',
 './output/US-MVW/01_qc_visual/qcv_files/US-MVW_qcv_201

### Below code are for debugging 

In [ ]:
data.iloc[-1]['TIMESTAMP_END'][:4]

In [ ]:
data.index[-1]

In [ ]:
start_day = '{:4d}01010000'.format(last_year)
end_day = '{:4d}01010000'.format(last_year + 1)


In [ ]:
data['TIMESTAMP_START'].count()

In [ ]:
T = pd.date_range(start_day,end_day,freq='30min')
T

In [ ]:
T.size,data.index[-1]

In [ ]:
new_index = pd.RangeIndex(T.size - data.index[-1]-2)

In [ ]:
new_df = pd.DataFrame(np.nan, index= new_index, columns=data.columns)
new_df

In [ ]:
start_t = data['TIMESTAMP_START'].count()
j = 0
for i in range(start_t, T.size-1):
    start_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i].year,T[i].month,T[i].day,T[i].hour, T[i].minute)
    end_time = '{:4d}{:02d}{:02d}{:02d}{:02d}'.format(T[i+1].year,T[i+1].month,T[i+1].day,T[i+1].hour, T[i+1].minute)
    new_df.loc[j,'TIMESTAMP_START'] = start_time
    new_df.loc[j,'TIMESTAMP_END'] = end_time
    j+=1

In [ ]:
new_df

In [ ]:
full_data = pd.concat([data,new_df], axis=0,ignore_index=True)
full_data

In [ ]:
_cols = data.columns[2:]
_cols

In [ ]:
full_data = full_data[_cols].interpolate()

In [ ]:
full_data